In [1]:

# !pip freeze | grep -v 'ipykernel' | xargs pip uninstall -y

# Check if there are packages besides ipykernel
result = !pip freeze | grep -v 'ipykernel\|ipython\|jupyter'
if result:
    print(f"Uninstalling {len(result)} packages...")
    !pip freeze | grep -v 'ipykernel\|ipython\|jupyter' | xargs pip uninstall -y
else:
    print("No packages to uninstall")

Uninstalling 89 packages...
Found existing installation: adam_atan2 0.0.3
Uninstalling adam_atan2-0.0.3:
  Successfully uninstalled adam_atan2-0.0.3
Found existing installation: annotated-types 0.7.0
Uninstalling annotated-types-0.7.0:
  Successfully uninstalled annotated-types-0.7.0
Found existing installation: antlr4-python3-runtime 4.9.3
Uninstalling antlr4-python3-runtime-4.9.3:
  Successfully uninstalled antlr4-python3-runtime-4.9.3
Found existing installation: argdantic 1.3.3
Uninstalling argdantic-1.3.3:
  Successfully uninstalled argdantic-1.3.3
Found existing installation: asttokens 3.0.0
Uninstalling asttokens-3.0.0:
  Successfully uninstalled asttokens-3.0.0
Found existing installation: certifi 2025.10.5
Uninstalling certifi-2025.10.5:
  Successfully uninstalled certifi-2025.10.5
Found existing installation: charset-normalizer 3.4.4
Uninstalling charset-normalizer-3.4.4:
  Successfully uninstalled charset-normalizer-3.4.4
Found existing installation: click 8.3.0
Uninstalling

In [ ]:
import os
print(os.getcwd())

!pip install --upgrade pip wheel setuptools
!pip install --pre --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu130 # install torch based on your cuda version
!pip install -r requirements.txt # install requirements
!wandb login /REDACTED/ # login if you want the logger to sync results to your Weights & Biases (https://wandb.ai/)


/home/alexfranks/Projects/9520/TinyRecursiveModels
Looking in indexes: https://download.pytorch.org/whl/nightly/cu130
  Using cached https://download.pytorch.org/whl/nightly/cu130/torchvision-0.25.0.dev20251025%2Bcu130-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (5.4 kB)
  Using cached https://download.pytorch.org/whl/nightly/filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached https://download.pytorch.org/whl/nightly/typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached https://download.pytorch.org/whl/nightly/sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached https://download.pytorch.org/whl/nightly/networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached https://download.pytorch.org/whl/nightly/jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached https://download.pytorch.org/whl/nightly/fsspec-2025.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached https://download.pytorch.org/whl/nightly/cu130/nvidia_cuda_nvrtc-13.0.

# Handle adam-atan2 issue

In [11]:
# !pip install adam-atan2 --no-binary adam-atan2
# !pip install --no-cache-dir --no-build-isolation adam-atan2
!pip uninstall adam-atan2
!pip install hatchling
!pip install git+https://github.com/lucidrains/adam-atan2-pytorch.git --no-build-isolation

  Using cached pluggy-1.6.0-py3-none-any.whl.metadata (4.8 kB)
Using cached pluggy-1.6.0-py3-none-any.whl (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [hatchling]
  Cloning https://github.com/lucidrains/adam-atan2-pytorch.git to /tmp/pip-req-build-n8_pnms8
  Running command git clone --filter=blob:none --quiet https://github.com/lucidrains/adam-atan2-pytorch.git /tmp/pip-req-build-n8_pnms8
  Resolved https://github.com/lucidrains/adam-atan2-pytorch.git to commit 0a753fdafe2438065bf3535a1c3ffb02d95d33f2
  Preparing metadata (pyproject.toml) ... done
  Created wheel for adam-atan2-pytorch: filename=adam_atan2_pytorch-0.2.4-py3-none-any.whl size=15032 sha256=404d1c5ba331633a6f6e395891ed28cb13a7cf622ac824ba9913b8286ad97e99
  Stored in directory: /tmp/pip-ephem-wheel-cache-b8_0jzmb/wheels/98/82/5c/c834ab67be38682b8827fbc0901d1affe1bbf3f9e97573d066
Successfully built adam-atan2-pytorch


# Dataset Preparation

In [3]:
# Sudoku-Extreme
!python dataset/build_sudoku_dataset.py --output-dir data/sudoku-extreme-1k-aug-1000  --subsample-size 1000 --num-aug 1000  # 1000 examples, 1000 augments

100%|██████████████████████████████| 422786/422786 [00:00<00:00, 2803940.06it/s]


# Experiments: Sudoku-Extreme (assuming 1 L40S GPU):

Runtime: < 36 hours

In [ ]:
!CUDA_LAUNCH_BLOCKING=1 TORCH_USE_CUDA_DSA=1 HYDRA_FULL_ERROR=1 run_name="pretrain_mlp_t_sudoku" python pretrain.py \
arch=trm \
data_paths="[data/sudoku-extreme-1k-aug-1000]" \
evaluators="[]" \
epochs=50000 eval_interval=5000 \
lr=1e-4 puzzle_emb_lr=1e-4 weight_decay=1.0 puzzle_emb_weight_decay=1.0 \
arch.mlp_t=True arch.pos_encodings=none \
arch.L_layers=2 \
arch.H_cycles=3 arch.L_cycles=6 \
+run_name=${run_name} ema=True

# !run_name="pretrain_att_sudoku" \
# python pretrain.py \
# arch=trm \
# data_paths="[data/sudoku-extreme-1k-aug-1000]" \
# evaluators="[]" \
# epochs=50000 eval_interval=5000 \
# lr=1e-4 puzzle_emb_lr=1e-4 weight_decay=1.0 puzzle_emb_weight_decay=1.0 \
# arch.L_layers=2 \
# arch.H_cycles=3 arch.L_cycles=6 \
# +run_name=${run_name} ema=True

/home/alexfranks/Projects/9520/TinyRecursiveModels/trm.venv/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/alexfranks/Projects/9520/TinyRecursiveModels/trm.venv/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` met